In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

import os
os.chdir("/content/gdrive/MyDrive/QuoraQuestions")
!ls

Mounted at /content/gdrive/
 CalculateCosineSimilarity.ipynb   modelA
 CleanData_Juliet.ipynb		   modelA_last.h5
 cleaned_features.csv		   modelB
 FindClusters.ipynb		   modelB_last.h5
 fulltrain_cleaned.csv		   modelC
 fulltrain.csv			   modelC_last.h5
 glove.6B.50d.txt		   modelD
 glove.twitter.27B.200d.txt	   mytokenizer.pickle
'Glove Vectors.ipynb'		   sample_submission.csv
 LSTM_Model_Inference.ipynb	   test.csv
 LSTM_training.ipynb		   train.csv


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
%matplotlib inline

In [ ]:
train_df = pd.read_csv("/content/gdrive/MyDrive/QuoraQuestions/cleaned_features.csv")
train_df = train_df.dropna()

train_df["qid1"].astype('int64', copy=False)
train_df["qid2"].astype('int64', copy=False)
train_df["question1_cleaned"].astype('string', copy=False)
train_df["question2_cleaned"].astype('string', copy=False)
train_df["is_duplicate"].astype('int64', copy=False)

print("total number of samples: ", len(train_df))
print("number of negative samples: " , len(train_df[train_df['is_duplicate'] == 0]))
print("number of positive samples: " , len(train_df[train_df['is_duplicate'] == 1]))

print(train_df.head())
print(train_df.columns)

total number of samples:  509792
number of negative samples:  254896
number of positive samples:  254896
   is_duplicate  qid1  qid2  ... Partial_Ratio Token_Sort_Ratio  Token_Set_Ratio
0             0     1     2  ...           100               93              100
1             0     3     4  ...            73               63               86
2             0     5     6  ...            53               66               66
3             0     7     8  ...            40               36               36
4             0     9    10  ...            55               47               67

[5 rows x 20 columns]
Index(['is_duplicate', 'qid1', 'qid2', 'question1_cleaned',
       'question2_cleaned', 'cosine_similarity', 'q1_word_count',
       'q2_word_count', 'q1char_count', 'q2char_count', 'freq_qid1',
       'freq_qid2', 'common_words_count', 'total_unique_num_words',
       'tot_words', 'words_ratio', 'Simple_Ratio', 'Partial_Ratio',
       'Token_Sort_Ratio', 'Token_Set_Ratio'],
      dt

In [ ]:
id_to_sentence = {}
for idx, sample in train_df.iterrows():
    sample1_id = sample['qid1']
    sample2_id = sample['qid2']

    if sample1_id not in id_to_sentence.keys():
        id_to_sentence[sample1_id] = sample['question1_cleaned']
    
    if sample2_id not in id_to_sentence.keys():
        id_to_sentence[sample2_id] = sample['question2_cleaned']

print(id_to_sentence[10])

which fish would survive in salt water


In [ ]:


def get_linking_dict(id_to_cluster, cluster_to_id, is_duplicate):
  newest_cluster = 0
  for idx, sample in train_df[train_df['is_duplicate'] == is_duplicate].iterrows():
    sample1_id = sample['qid1']
    sample2_id = sample['qid2']

    empty_clusters = []

    if sample1_id in id_to_cluster and sample2_id in id_to_cluster:
      if id_to_cluster[sample1_id] != id_to_cluster[sample2_id]:
        empty_clusters.append(id_to_cluster[sample2_id])
        new_id = id_to_cluster[sample1_id]
        remove_id = id_to_cluster[sample2_id]

        for id_to_change in cluster_to_id[remove_id]:
          id_to_cluster[id_to_change] = new_id
        
        cluster_to_id[new_id] = cluster_to_id[new_id] + cluster_to_id[remove_id]
        cluster_to_id[remove_id] = []
      else:
        pass
    elif sample1_id in id_to_cluster:
      current_cluster = id_to_cluster[sample1_id]
      id_to_cluster[sample2_id] = current_cluster
      cluster_to_id[current_cluster].append(sample2_id)
    elif sample2_id in id_to_cluster:
      current_cluster = id_to_cluster[sample2_id]
      id_to_cluster[sample1_id] = current_cluster
      cluster_to_id[current_cluster].append(sample1_id)
    else:
      if empty_clusters:
        popped = empty_clusters.pop()
        id_to_cluster[sample1_id] = popped
        id_to_cluster[sample2_id] = popped
        cluster_to_id[popped] = [sample1_id, sample2_id]
      else:
        id_to_cluster[sample1_id] = newest_cluster
        id_to_cluster[sample2_id] = newest_cluster
        cluster_to_id[newest_cluster] = [sample1_id, sample2_id]
        newest_cluster = newest_cluster + 1
  print(empty_clusters)
  return newest_cluster

id_to_cluster_duplicate = {}
cluster_to_id_duplicate = {}

id_to_cluster_nonduplicate = {}
cluster_to_id_nonduplicate = {}

starting_cluster_number = get_linking_dict(id_to_cluster_duplicate, cluster_to_id_duplicate, 1)
get_linking_dict(id_to_cluster_nonduplicate, cluster_to_id_nonduplicate, 0)

[]
[]


173419

In [ ]:
def find_top_k_cluster(cluster_to_id, k):
  x = []
  for cluster_key in cluster_to_id.keys():
    length = len(cluster_to_id[cluster_key])
    mytuple = (length, cluster_key)
    x.append(mytuple)
  x.sort(reverse=True)

  for size, cluster_num in x[0:k]:
    print("size:", size, "cluster_id:", cluster_num)

find_top_k_cluster(cluster_to_id_duplicate,80)

size: 109 cluster_id: 32952
size: 97 cluster_id: 9952
size: 97 cluster_id: 1254
size: 85 cluster_id: 14527
size: 76 cluster_id: 3144
size: 74 cluster_id: 10546
size: 70 cluster_id: 31584
size: 58 cluster_id: 8980
size: 58 cluster_id: 801
size: 54 cluster_id: 12
size: 53 cluster_id: 1425
size: 51 cluster_id: 7982
size: 51 cluster_id: 1565
size: 49 cluster_id: 689
size: 47 cluster_id: 10276
size: 47 cluster_id: 8056
size: 44 cluster_id: 10285
size: 43 cluster_id: 15440
size: 42 cluster_id: 8833
size: 40 cluster_id: 3927
size: 38 cluster_id: 15730
size: 38 cluster_id: 11636
size: 38 cluster_id: 7675
size: 37 cluster_id: 33421
size: 37 cluster_id: 6425
size: 37 cluster_id: 444
size: 36 cluster_id: 8344
size: 36 cluster_id: 4022
size: 35 cluster_id: 8595
size: 35 cluster_id: 4611
size: 35 cluster_id: 4356
size: 33 cluster_id: 16937
size: 33 cluster_id: 9630
size: 33 cluster_id: 5054
size: 33 cluster_id: 3822
size: 32 cluster_id: 32745
size: 32 cluster_id: 25389
size: 32 cluster_id: 7144
siz

In [ ]:
for qid in cluster_to_id_duplicate[9952]:
  print(id_to_sentence[qid])

what do you think about the idea of narendra modi about 500 and 1 thousand notes
what do you think about narendra modi s decision about banning 500rs and 1 thousand rs notes and what will be the effects do this make corruption free india
what does balaji vishwanathan think about the ban of rupee 500 and rupee 1 thousand currency notes in india
what do you think is the reason for pm modi s instant decision to ban rupee 500 and rupee 1 thousand notes
what as your reaction when your heard about the scraping currency notes of inr 500 and 1 thousand
what is your opinion about narendra modis banning on 500 and 1 thousand
what are you views pros cons on ban of 500 and 1 thousand notes by goverment
do not you think modi will repeal decision to ban 500 and 1 thousand notes as it will cause high inconvenience
what is your reaction about the ban on rs 500 and rs 1 thousand notes will not it create a chaos and harm the economy
what do you think about the ban on 500 and 1 thousand denomination note

In [ ]:
find_top_k_cluster(cluster_to_id_nonduplicate,30)

size: 5674 cluster_id: 103021
size: 842 cluster_id: 3149
size: 624 cluster_id: 90592
size: 362 cluster_id: 82944
size: 328 cluster_id: 54931
size: 321 cluster_id: 149471
size: 275 cluster_id: 111836
size: 254 cluster_id: 23617
size: 247 cluster_id: 113697
size: 243 cluster_id: 27340
size: 242 cluster_id: 10882
size: 211 cluster_id: 15241
size: 207 cluster_id: 79751
size: 188 cluster_id: 74623
size: 182 cluster_id: 14320
size: 161 cluster_id: 149282
size: 159 cluster_id: 17207
size: 156 cluster_id: 30510
size: 139 cluster_id: 9835
size: 135 cluster_id: 129452
size: 135 cluster_id: 113315
size: 124 cluster_id: 71340
size: 120 cluster_id: 122492
size: 106 cluster_id: 115727
size: 104 cluster_id: 106446
size: 98 cluster_id: 108220
size: 97 cluster_id: 84707
size: 96 cluster_id: 47606
size: 91 cluster_id: 74102
size: 91 cluster_id: 2676


In [ ]:
for qid in cluster_to_id_nonduplicate[2676]:
  print(id_to_sentence[qid])

how can you determine the lewis structure for dimethylamine
how is the lewis structure for ch3o determined
how can you determine the lewis structure for co
how can you determine the lewis structure for no
how do you determine the lewis structure for hclo3
how can you determine the lewis structure for sf5
how can you determine the lewis structure for scl6
how can you determine the lewis structure of ch2oh
how do you determine the lewis structure of ch3och3
how can the lewis structure for cof2 be determined
how is the lewis structure for c3h6 determined
how can the lewis structure for ch2chch3 be determined
how do you determine the lewis structure of c2h2
how do you determine the lewis structure of cl2co
how do you determine the lewis structure for h2c2o4
how can you determine the lewis structure for ch3cn
how is the lewis structure for pf3cl2 determined
how can you determine the lewis structure for ch20
how is the lewis structure for brcn determined
how can the lewis structure for nh2nh

In [ ]:
def save_cluster(cluster_to_id, filename):
  x = {}
  for cluster_key in cluster_to_id.keys():
    length = len(cluster_to_id[cluster_key])

    x[cluster_key] = {"id_list": cluster_to_id[cluster_key], "length": length}

  with open(filename, 'w') as fp:
    json.dump(x, fp, sort_keys=True, indent=4)
  
  return x

nd_dict = save_cluster(cluster_to_id_nonduplicate, "/content/gdrive/MyDrive/QuoraQuestions/non_duplicate_cluster.json")


In [ ]:
d_dict = save_cluster(cluster_to_id_duplicate, "/content/gdrive/MyDrive/QuoraQuestions/duplicate_cluster.json")